In [316]:
import numpy as np
import random
import time

# This cell does the setup for the rest. 
# Only the first two variables should be changed by the user!


m=8
dist=15

n=2**m-1

def cyclotomic_coset(j):
    i=j
    ccs={j}
    while (i*2)%n not in ccs:
        i=(i*2)%n
        ccs=ccs.union({i})
    return ccs

def narrow_sense_T():
    T=set()
    for j in range(1,dist):
        T=T.union(cyclotomic_coset(j))
    return T

T=narrow_sense_T()
k=n-len(T)

print("n",n)
print("k",k)
print("|T|",len(T))

F=GF(2)
E=GF(2**m,'w')
R.<X>=PolynomialRing(F)
alpha=E.gen()

n 255
k 199
|T| 56


In [317]:
def generator_polynomial():
    return prod([X-alpha**i for i in T])

def encode_message(message, g):
    r=(R(message)*X**(n-k))%g
    codeword=r.list()+([0]*(g.degree()-r.degree()-1))+message
    return codeword

In [318]:
#auxillary functions for decoding

def syndromes(received_polynomial):
    return [received_polynomial(alpha**i) for i in range(1,dist)]

def bm(s):
    X = R.0
    C = R.one()
    B = R.one()
    L = 0
    deltap = 1
    m = -1
    for j in range(len(s)):
        delta = sum([ s[j-i]*C[i] for i in range(C.degree()+1)])
        if delta != 0:
            T = C
            C = C - delta/deltap*X^(j-m)*B
            if 2*L <= j:
                B = T
                m = j
                deltap = delta
                L = j + 1 - L
        #print(j,"|",L,"|",C,"|",B,"|",delta)
    return (L,C)

def error_locations(C):
    return [(n-i)%n for i in range(n) if C(alpha**i)==0]

In [325]:
def decode_codeword(codeword,g):
    s=syndromes(R(codeword))
    L,C=bm(s)
    error_size=C.degree()
    for i in error_locations(C):
        codeword[i]=(Integer(codeword[i])+1)%2
    return codeword[n-k:]

    

m=[1]*k
m[2]=0
m[16]=0
m[0]=0
g=generator_polynomial()
codeword=encode_message(m,g)
error=[0]*(n)
error[3]=1
error[5]=1
Z=list(np.array(codeword)+np.array(error))
print(m)
print(decode_codeword(Z,g))

[0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [324]:
g=generator_polynomial()
for blub in range(5):
    message=[0]*k
    for i in random.sample(range(k), k-2):
            message[i]=1
    codeword=encode_message(message,g)
    
    for bla in range(2):
        error=[0]*(n)
        for i in random.sample(range(n), (dist-1)//2):
            codeword[i]=(Integer(codeword[i])+1)%2
        tic=time.perf_counter()
        decoded_message=decode_codeword(codeword,g)
        toc=time.perf_counter()
        print(toc-tic)
        if not decoded_message==message:
            print("Fail")

0.08700589998625219
0.08159519999753684
0.09890949999680743
0.09430900000734255
0.10893950000172481
0.09437529998831451
0.09317150001879781
0.1071407999843359
0.09560830000555143
0.09485799999674782


In [286]:
print(4//3)

1
